In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

In [5]:
# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

In [6]:
server = hook.local_worker

In [7]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

In [8]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

In [9]:
plan_double_abs

<Plan plan_double_abs id:68674798427 owner:me>

To use a plan, you need two things: 
1. To build the plan (ie register the sequence of operations present in the function) 
2. To send it to a worker / device

In [11]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

tensor([-1.,  2.])
	Tags: input_data 
	Shape: torch.Size([2])

In [12]:
plan_double_abs.is_built

False

In [13]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


In [14]:
plan_double_abs.build(torch.tensor([1., -2.]))

PlaceHolder[Id:45843229931]>tensor([2., 4.])

In [15]:
plan_double_abs.is_built

True

In [16]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:13673574868 -> device_1:68674798427]

In [17]:
# 注意在line 11中有pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

(Wrapper)>[PointerTensor | me:34898527106 -> device_1:44471190935]


In [18]:
pointer_to_result.get()

tensor([2., 4.])

In [19]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [20]:
net = Net()

In [21]:
net

<Net Net id:2836269152 owner:me>

In [23]:
net.build(torch.tensor([1., 2.]))

/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py:296: UserWarning: Failed to translate Plan with PlanTranslatorTfjs: Traceback (most recent call last):
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py", line 292, in build
    self.add_translation(translator)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py", line 503, in add_translation
    role = plan_translator(self).translate()
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 116, in translate
    return self.translate_framework(TranslationTarget.TENSORFLOW_JS.value)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 102, in translate_framework
    self.translate_action(action, to_framework, new_role)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 75, in translate_action
    translated_cmds = threepio.translate(Command(function_name, args, action.kwargs))
  File "/root/gsq_workplace/a

PlaceHolder[Id:97524080203]>tensor([-0.7597, -0.6307], grad_fn=<LogSoftmaxBackward>)

In [24]:
pointer_to_net = net.send(device_1)
pointer_to_net

[PointerPlan | me:59184189690 -> device_1:2836269152]

In [25]:
pointer_to_data = device_1.search('input_data')[0]

In [26]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:92764518390 -> device_1:44117132467]

In [27]:
pointer_to_result.get()

tensor([-0.5570, -0.8509], requires_grad=True)

In [28]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [29]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py:296: UserWarning: Failed to translate Plan with PlanTranslatorTfjs: Traceback (most recent call last):
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py", line 292, in build
    self.add_translation(translator)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/plan.py", line 503, in add_translation
    role = plan_translator(self).translate()
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 116, in translate
    return self.translate_framework(TranslationTarget.TENSORFLOW_JS.value)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 102, in translate_framework
    self.translate_action(action, to_framework, new_role)
  File "/root/gsq_workplace/arinn/PySyft/syft/execution/translation/threepio.py", line 75, in translate_action
    translated_cmds = threepio.translate(Command(function_name, args, action.kwargs))
  File "/root/gsq_workplace/a

PlaceHolder[Id:37567234217]>tensor([-0.8292, -0.5734], grad_fn=<LogSoftmaxBackward>)

In [30]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

tensor([-0.8292, -0.5734], requires_grad=True)

In [31]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

tensor([-0.8292, -0.5734], requires_grad=True)

In [32]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

True

In [33]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

True

In [34]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [35]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

tensor([2, 1])